***GENERATED CODE FOR eligdenresolutionrecommendationautoml PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = transformationDF.drop('ReasonDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '92323.29', 'stddev': '326571.84', 'min': '11042', 'max': 'S9083', 'missing': '0', 'distinct': '123'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '5.07', 'stddev': '11.01', 'min': '0000', 'max': 'QW', 'missing': '0', 'distinct': '22'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier'}, {'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier2', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '0.09', 'stddev': '2.35', 'min': '0000', 'max': 'LT', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier2'}, {'feature_label': 'CptModifier2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'A04.72', 'max': 'Z95.4', 'missing': '0', 'distinct': '385'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'A09', 'max': 'Z99.81', 'missing': '0', 'distinct': '406'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'U', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'Cardiac Cath', 'max': 'Total Eye', 'missing': '0', 'distinct': '30'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'Cardiac Surgery', 'max': 'Vascular Surgery', 'missing': '0', 'distinct': '24'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SpecialityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'Aetna US Healthcare', 'max': 'Work Comp', 'missing': '0', 'distinct': '16'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerCategory'}, {'feature_label': 'PrimaryPayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Work Comp', 'missing': '0', 'distinct': '13'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerGroup'}, {'feature_label': 'PrimaryPayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerName', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'AARP 2ndary', 'max': 'Work Comp', 'missing': '0', 'distinct': '68'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerName'}, {'feature_label': 'PrimaryPayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerSubGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerSubGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'United Healthcare Medicare Advantage', 'missing': '0', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerSubGroup1'}, {'feature_label': 'PrimaryPayerSubGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerSubGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerSubGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerSubGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Workers Comp', 'missing': '0', 'distinct': '23'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerSubGroup2'}, {'feature_label': 'PrimaryPayerSubGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerSubGroup2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'Aetna US Healthcare', 'max': 'United Healthcare', 'missing': '0', 'distinct': '14'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerCategory'}, {'feature_label': 'SecondPayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'UHC', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerGroup'}, {'feature_label': 'SecondPayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerName', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'AARP 2ndary', 'max': 'Wellcare Staywell', 'missing': '0', 'distinct': '47'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerName'}, {'feature_label': 'SecondPayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerSubGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerSubGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'AvMed', 'max': 'United Healthcare Medicare Advantage', 'missing': '0', 'distinct': '14'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerSubGroup1'}, {'feature_label': 'SecondPayerSubGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerSubGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerSubGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerSubGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Wellcare/Staywell', 'missing': '0', 'distinct': '21'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerSubGroup2'}, {'feature_label': 'SecondPayerSubGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerSubGroup2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceItemName', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceItemName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '92323.29', 'stddev': '326571.84', 'min': '11042', 'max': 'S9083', 'missing': '0', 'distinct': '123'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceItemName'}, {'feature_label': 'ServiceItemName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceItemName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceItemCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceItemCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': '3D RENDER W/O POSTPROCESS', 'max': 'Zinc paste band w >=3"<5"/yd', 'missing': '0', 'distinct': '114'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceItemCategory'}, {'feature_label': 'ServiceItemCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceItemCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceLocation', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceLocation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'Home', 'max': 'Urgent Care Facility', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceLocation'}, {'feature_label': 'ServiceLocation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceLocation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'reasonType', 'transformation_label': 'String Indexer'}], 'feature': 'reasonType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'Denial', 'max': 'Denial', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'reasonType'}, {'feature_label': 'reasonType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('reasonType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'}], 'feature': 'Reasoncode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '32.22', 'stddev': '25.57', 'min': '109', 'max': '33', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Reasoncode'}, {'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Reasoncode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Remarkcode', 'transformation_label': 'String Indexer'}], 'feature': 'Remarkcode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'CO109', 'max': 'PR33', 'missing': '0', 'distinct': '22'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Remarkcode'}, {'feature_label': 'Remarkcode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Remarkcode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': 'Benefits Max/Exh', 'max': 'Eligibility', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonCategory'}, {'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonDescription', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '1261', 'mean': '', 'stddev': '', 'min': 'Charges are covered under a capitation agreement/managed care plan.',
                                                                                                                                                                                                                                                                                    'max': 'This is a work-related injury/illness and thus the liability of the Worker`s Compensation Carrier.', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonDescription'}, {'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonDescription')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Age_Bins', 'transformation_label': 'String Indexer'}], 'feature': 'Age_Bins', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': '31-50', 'max': '<=30', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Age_Bins'}, {'feature_label': 'Age_Bins', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Age_Bins')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender_Age_Bins', 'transformation_label': 'String Indexer'}], 'feature': 'Gender_Age_Bins', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender_Age_Bins'}, {'feature_label': 'Gender_Age_Bins', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender_Age_Bins')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Amount_Bins', 'transformation_label': 'String Indexer'}], 'feature': 'Amount_Bins', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Amount_Bins'}, {'feature_label': 'Amount_Bins', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Amount_Bins')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'label_resolution', 'transformation_label': 'String Indexer'}], 'feature': 'label_resolution', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1261', 'mean': '27.46', 'stddev': '10.62', 'min': '0', 'max': '64', 'missing': '0', 'distinct': '25'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'label_resolution'}, {'feature_label': 'label_resolution', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('label_resolution')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run eligdenresolutionrecommendationautomlHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_BOND_CLINIC', 'optionalDB': 'NextGen_BOND_CLINIC', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT\r\n\t\r\n\t\t\t\tcd.ChargeDetailID, \r\n\t\t\t\ttr.ReasonDate,\r\n\t\t\t\tcptc.CPTCode,\r\n\t\t\t\tCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\n\t\t\t\tCASE WHEN cd.CPTModifier2 IS NOT NULL THEN cd.CPTModifier2 ELSE '0000' END as CptModifier2,\r\n\t\t\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\t\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\t\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\n\t            cd.Amount,\r\n\t\t\t\td.DepartmentCategory,\r\n\t\t\t\ts.SpecialityCategory,\r\n\r\n\t\t\t\tCASE WHEN py.PayerCategory IS NOT NULL THEN py.PayerCategory ELSE 'NOT AVAILABLE' END as PrimaryPayerCategory ,\r\n\t\t\t\tCASE WHEN py.PayerGroup IS NOT NULL THEN py.PayerGroup ELSE 'NOT AVAILABLE' END as PrimaryPayerGroup ,\r\n\t\t\t\tCASE WHEN py.PayerName IS NOT NULL THEN py.PayerName ELSE 'NOT AVAILABLE' END as PrimaryPayerName ,\r\n\t\t\t\tCASE WHEN py.PayerSubGroup1 IS NOT NULL THEN py.PayerSubGroup1 ELSE 'NOT AVAILABLE' END as PrimaryPayerSubGroup1 ,\r\n\t\t\t\tCASE WHEN py.PayerSubGroup2 IS NOT NULL THEN py.PayerSubGroup2 ELSE 'NOT AVAILABLE' END as PrimaryPayerSubGroup2 ,\r\n\t\r\n\t\t\t\tCASE WHEN py1.PayerCategory IS NOT NULL THEN py1.PayerCategory ELSE 'NOT AVAILABLE' END as SecondPayerCategory ,\r\n\t\t\t\tCASE WHEN py1.PayerGroup IS NOT NULL THEN py1.PayerGroup ELSE 'NOT AVAILABLE' END as SecondPayerGroup ,\r\n\t\t\t\tCASE WHEN py1.PayerName IS NOT NULL THEN py1.PayerName ELSE 'NOT AVAILABLE' END as SecondPayerName ,\r\n\t\t\t\tCASE WHEN py1.PayerSubGroup1 IS NOT NULL THEN py1.PayerSubGroup1 ELSE 'NOT AVAILABLE' END as SecondPayerSubGroup1 ,\r\n\t\t\t\tCASE WHEN py1.PayerSubGroup2 IS NOT NULL THEN py1.PayerSubGroup2 ELSE 'NOT AVAILABLE' END as SecondPayerSubGroup2 ,\r\n\t\r\n\t\t\t\tCASE WHEN si.ServiceItemName IS NOT NULL THEN si.ServiceItemName ELSE 'NOT AVAILABLE' END as ServiceItemName ,\r\n\t\t\t\tCASE WHEN si.ServiceItemCategory IS NOT NULL THEN si.ServiceItemCategory ELSE 'NOT AVAILABLE' END as ServiceItemCategory,\r\n\t\t\t\tCASE WHEN sl.ServiceLocation IS NOT NULL THEN sl.ServiceLocation ELSE 'NOT AVAILABLE' END as ServiceLocation,\r\n\r\n\t\t\ttrc.reasonType,\r\n\t\t\ttrc.TransactionReasonCodeID,\r\n\t\t\ttrc.Reasoncode,\r\n\t\t\ttr.SourceKey as Remarkcode,\r\n\t\t\ttrc.ReasonCategory,\r\n\t\t\ttrc.ReasonDescription,\r\n\t\t\t\r\n\t\t\t\tCASE WHEN sl.ServiceLocation IS NULL THEN 1 ELSE 0 END as NoServiceLocation,\r\n\t\t\t\tDATEPART(WEEKDAY, tr.ReasonDate) as ReasonDay, \r\n\t\t\t\tDATEPART(MONTH, tr.ReasonDate) as ReasonMonth, \r\n\r\n\r\n\t\t\t\tCASE WHEN FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) >= 0 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=30 THEN '<=30' \r\n\t\t\t\t\t WHEN FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 30 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=50 THEN '31-50'\r\n\t\t\t\t\t WHEN FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 50 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=70 THEN '51-70'\r\n\t\t\t\t\t WHEN FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 70 THEN '70+' END AS Age_Bins,\r\n\r\n\r\n\t\t\t\tCASE WHEN p.Gender='M' and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) >= 0 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=30 THEN 'Male - <=30' \r\n\t\t\t\t\t WHEN p.Gender='M' and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 30 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=50 THEN 'Male - 31-50'\r\n\t\t\t\t\t WHEN p.Gender='M' and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 50 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=70 THEN 'Male - 51-70'\r\n\t\t\t\t\t WHEN p.Gender='M' and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 70 THEN 'Male - 70+'  \r\n\t\t\t\t\t WHEN p.Gender='F' and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) >= 0 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=30 THEN 'Female - <30' \r\n\t\t\t\t\t WHEN p.Gender='F' and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 30 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=50 THEN 'Female - 31-50'\r\n\t\t\t\t\t WHEN p.Gender='F' and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 50 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=70 THEN 'Female - 51-70'\r\n\t\t\t\t\t WHEN p.Gender='F' and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 70 THEN 'Female - 70+' \r\n\t\t\t\t\t ELSE 'Other gender' END AS Gender_Age_Bins, \r\n\r\n\t\t\t\tCASE WHEN cd.Amount >= 0 and cd.Amount <=100 THEN '<=100' \r\n\t\t\t\t\t WHEN cd.Amount > 100 and cd.Amount <=200 THEN '101-200'\r\n\t\t\t\t\t WHEN cd.Amount > 200 and cd.Amount <=300 THEN '201-300'\r\n\t\t\t\t\t WHEN cd.Amount > 300 and cd.Amount <=400 THEN '301-400'\r\n\t\t\t\t\t WHEN cd.Amount > 400 and cd.Amount <=500 THEN '401-500' \r\n\t\t\t\t\t WHEN cd.Amount > 500 THEN '500+' END AS Amount_Bins, \r\n\r\n\t\t\t\tCASE WHEN cd.payerid IS NOT NULL THEN 1 ELSE 0 END as PrimaryPayerAvailable ,\r\n\t\t\t\tCASE WHEN cd.SecondaryPayerid IS NOT NULL THEN 1 ELSE 0 END as SecondaryPayerAvailable ,\r\n\t\t\t\tCASE WHEN cd.TertiaryPayerid IS NOT NULL THEN 1 ELSE 0 END as TertiaryPayerAvailable ,\r\n\r\n\t\t\t\tCASE WHEN cd.payerid IS NOT NULL and cd.SecondaryPayerid IS NOT NULL and cd.TertiaryPayerid IS NOT NULL THEN 3\r\n\t\t\t\t     WHEN cd.payerid IS NOT NULL and cd.SecondaryPayerid IS NOT NULL and cd.TertiaryPayerid IS NULL THEN 2\r\n\t\t\t\t     WHEN cd.payerid IS NOT NULL and cd.SecondaryPayerid IS NULL and cd.TertiaryPayerid IS NOT NULL THEN 2\r\n\t\t\t\t     WHEN cd.payerid IS NOT NULL and cd.SecondaryPayerid IS NULL and cd.TertiaryPayerid IS NULL THEN 1\r\n\t\t\t\t\t ELSE 0 END as NumOfPayers,\r\n\r\n\t\t\t\tCASE WHEN py.PayerCategory ='Self Pay' or py.PayerCategory ='Cash Account' THEN 1 ELSE 0 END as IsPrimarySelfPay,\r\n\t\t\t\tCASE WHEN py.PayerCategory ='Non Contracted' THEN 1 ELSE 0 END as IsPrimaryNonContracted\r\n\r\n\t,CASE \r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '3'  THEN '1'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '4'  THEN '2'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '5'  THEN '3'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '6'  THEN '4'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '7'  THEN '5'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '8'  THEN '6'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '9'  THEN '7'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '10'  THEN '8'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '11'  THEN '9'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '12'  THEN '10'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '13'  THEN '11'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '14'  THEN '12'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '15'  THEN '13'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '16'  THEN '14'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '41'  THEN '15'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '42'  THEN '16'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '43'  THEN '17'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '44'  THEN '18'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '45'  THEN '19'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '46'  THEN '20'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '47'  THEN '21'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '48'  THEN '22'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '49'  THEN '23'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '50'  THEN '24'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '51'  THEN '25'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '52'  THEN '26'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '53'  THEN '27'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '54'  THEN '28'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '55'  THEN '29'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '56'  THEN '30'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '57'  THEN '31'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '58'  THEN '32'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '59'  THEN '33'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '60'  THEN '34'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '61'  THEN '35'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '62'  THEN '36'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '63'  THEN '37'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '64'  THEN '38'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '65'  THEN '39'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '66'  THEN '40'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '67'  THEN '41'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '68'  THEN '42'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '69'  THEN '43'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '70'  THEN '44'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '71'  THEN '45'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '72'  THEN '46'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '73'  THEN '47'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '74'  THEN '48'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '75'  THEN '49'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '76'  THEN '50'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '77'  THEN '51'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '78'  THEN '52'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '79'  THEN '53'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '80'  THEN '54'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '81'  THEN '55'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '82'  THEN '56'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '83'  THEN '57'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '84'  THEN '58'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '85'  THEN '59'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '86'  THEN '60'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '87'  THEN '61'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '88'  THEN '62'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '89'  THEN '63'\r\n        WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '90'  THEN '64'\r\n\r\n\t\tWHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '789'  THEN '65'\r\n\t\tWHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '28512'  THEN '66'\r\n\t\tWHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '30899'  THEN '67'\r\n\t\tWHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = '34271'  THEN '68'\r\n\t\tELSE '0' END AS label_resolution\r\n\r\n\r\n\tfrom ChargeDetail cd\r\n\tinner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\n\tinner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID  \r\n\tleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = t.PayerID \r\n\tinner join Bill b WITH (NOLOCK) on e.EncounterID = b.EncounterID and b.BillID = t.BillID \r\n\tinner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID AND T.TransactionDate>=TR.ReasonDate\r\n\tinner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\n\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\tinner join Payer py WITH (NOLOCK) on py.PayerID = t.PayerID\r\n\tleft join Payer py1 WITH (NOLOCK) on py1.PayerID = cd.SecondaryPayerID\r\n\tleft join Payer py2 WITH (NOLOCK) on py2.PayerID = cd.TertiaryPayerID\r\n\tinner join ServiceItem si WITH (NOLOCK) on cd.ServiceItemID = si.ServiceItemID\r\n\tleft outer join ServiceLocation sl WITH (NOLOCK) on cd.ServiceLocationID = sl.ServiceLocationID\r\n\t\t\t\r\n\t\t\tinner join Transactiontype tt on tt.transactiontypeid = t.transactiontypeid\r\n\t\t\tleft outer join  Task tsk WITH (NOLOCK) on e.encounterid = tsk.encounterid and e.facilityid = tsk.facilityid and tsk.createddate >= tr.reasondate \r\n\t\t\tleft outer join  TaskCompletionReason tcr WITH (NOLOCK) on tsk.TaskCompletionReasonID = tcr.TaskCompletionReasonID  \r\n\t\t\tleft outer join  Taskstatus ts on tsk.TaskStatusID=ts.TaskStatusID\r\n\t\t\tleft outer join  TaskType tskt on tsk.TaskTypeID=tskt.TaskTypeID\r\n\r\nWHERE\t\tcd.Amount >.01\r\n\t\t\tand trc.ReasonCode IN ('19','20','21','22','24','26','27','28','30','31','32','33','109','166','200')\r\n\t\t\tand trc.reasonType = 'Denial'\r\n\t\t\tand tr.ReasonDate between '2018-01-01' and '2020-12-31'\r\n\t\t\tand ts.TaskStatusCategory ='Completed'\r\n\t\t\tand tcr.TaskCompletionReason !='NULL' \r\n\t\t\tand tcr.TaskCompletionReasonID NOT IN ('99','100','101') \r\n\t\t\tand tsk.CreatedDate >= tr.ReasonDate\r\n\t\t\tand TransactionDate >= tsk.CompletedDate \r\n\t\t\tand tskt.TaskTypeCategory='AR'\r\n\t\t\tand py.PayerCategory!='Self Pay'", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run eligdenresolutionrecommendationautomlHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "0026D1E0-5563-4310-85C9-B455F7090F23", "max": "FFDD4F75-F7AA-495B-8032-B3435F8E362F", "missing": "0", "distinct": "1117"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"feature_label": "ReasonDate", "transformation_label": "Extract Date"}], "feature": "ReasonDate", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "ReasonDate"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "92323.29", "stddev": "326571.84", "min": "11042", "max": "S9083", "missing": "0", "distinct": "123"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "CptModifier", "transformation_label": "String Indexer"}], "feature": "CptModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "5.07", "stddev": "11.01", "min": "0000", "max": "QW", "missing": "0", "distinct": "22"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier"}, {"transformationsData": [{"feature_label": "CptModifier2", "transformation_label": "String Indexer"}], "feature": "CptModifier2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "0.09", "stddev": "2.35", "min": "0000", "max": "LT", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier2"}, {"transformationsData": [{"feature_label": "pdiagnosis1", "transformation_label": "String Indexer"}], "feature": "pdiagnosis1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "A04.72", "max": "Z95.4", "missing": "0", "distinct": "385"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis1"}, {"transformationsData": [{"feature_label": "pdiagnosis2", "transformation_label": "String Indexer"}], "feature": "pdiagnosis2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "A09", "max": "Z99.81", "missing": "0", "distinct": "406"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1261", "mean": "71.36", "stddev": "15.4", "min": "3", "max": "105", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "F", "max": "U", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1261", "mean": "147.7", "stddev": "301.99", "min": "0.33", "max": "8617.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "Cardiac Cath", "max": "Total Eye", "missing": "0", "distinct": "30"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "SpecialityCategory", "transformation_label": "String Indexer"}], "feature": "SpecialityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "Cardiac Surgery", "max": "Vascular Surgery", "missing": "0", "distinct": "24"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SpecialityCategory"}, {"transformationsData": [{"feature_label": "PrimaryPayerCategory", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "Aetna US Healthcare", "max": "Work Comp", "missing": "0", "distinct": "16"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerCategory"}, {"transformationsData": [{"feature_label": "PrimaryPayerGroup", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "Aetna", "max": "Work Comp", "missing": "0", "distinct": "13"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerGroup"}, {"transformationsData": [{"feature_label": "PrimaryPayerName", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "AARP 2ndary", "max": "Work Comp", "missing": "0", "distinct": "68"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerName"}, {"transformationsData": [{"feature_label": "PrimaryPayerSubGroup1", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerSubGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "Aetna", "max": "United Healthcare Medicare Advantage", "missing": "0", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerSubGroup1"}, {"transformationsData": [{"feature_label": "PrimaryPayerSubGroup2", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerSubGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "Aetna", "max": "Workers Comp", "missing": "0", "distinct": "23"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerSubGroup2"}, {"transformationsData": [{"feature_label": "SecondPayerCategory", "transformation_label": "String Indexer"}], "feature": "SecondPayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "Aetna US Healthcare", "max": "United Healthcare", "missing": "0", "distinct": "14"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerCategory"}, {"transformationsData": [{"feature_label": "SecondPayerGroup", "transformation_label": "String Indexer"}], "feature": "SecondPayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "Aetna", "max": "UHC", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerGroup"}, {"transformationsData": [{"feature_label": "SecondPayerName", "transformation_label": "String Indexer"}], "feature": "SecondPayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "AARP 2ndary", "max": "Wellcare Staywell", "missing": "0", "distinct": "47"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerName"}, {"transformationsData": [{"feature_label": "SecondPayerSubGroup1", "transformation_label": "String Indexer"}], "feature": "SecondPayerSubGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "AvMed", "max": "United Healthcare Medicare Advantage", "missing": "0", "distinct": "14"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerSubGroup1"}, {"transformationsData": [{"feature_label": "SecondPayerSubGroup2", "transformation_label": "String Indexer"}], "feature": "SecondPayerSubGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "Aetna", "max": "Wellcare/Staywell", "missing": "0", "distinct": "21"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerSubGroup2"}, {"transformationsData": [{"feature_label": "ServiceItemName", "transformation_label": "String Indexer"}], "feature": "ServiceItemName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "92323.29", "stddev": "326571.84", "min": "11042", "max": "S9083", "missing": "0", "distinct": "123"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceItemName"}, {"transformationsData": [{"feature_label": "ServiceItemCategory", "transformation_label": "String Indexer"}], "feature": "ServiceItemCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "3D RENDER W/O POSTPROCESS", "max": "Zinc paste band w >=3\"<5\"/yd", "missing": "0", "distinct": "114"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceItemCategory"}, {"transformationsData": [{"feature_label": "ServiceLocation", "transformation_label": "String Indexer"}], "feature": "ServiceLocation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "Home", "max": "Urgent Care Facility", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceLocation"}, {"transformationsData": [{"feature_label": "reasonType", "transformation_label": "String Indexer"}], "feature": "reasonType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "Denial", "max": "Denial", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "reasonType"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TransactionReasonCodeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1261", "mean": "1149.77", "stddev": "86.66", "min": "884", "max": "1244", "missing": "0"}, "updatedLabel": "TransactionReasonCodeID"}, {"transformationsData": [{"feature_label": "Reasoncode", "transformation_label": "String Indexer"}], "feature": "Reasoncode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "32.22", "stddev": "25.57", "min": "109", "max": "33", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Reasoncode"}, {"transformationsData": [{"feature_label": "Remarkcode", "transformation_label": "String Indexer"}], "feature": "Remarkcode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "CO109", "max": "PR33", "missing": "0", "distinct": "22"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Remarkcode"}, {"transformationsData": [{"feature_label": "ReasonCategory", "transformation_label": "String Indexer"}], "feature": "ReasonCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "Benefits Max/Exh", "max": "Eligibility", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonCategory"}, {"transformationsData": [{"feature_label": "ReasonDescription", "transformation_label": "String Indexer"}], "feature": "ReasonDescription", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "Charges are covered under a capitation agreement/managed care plan.", "max": "This is a work-related injury/illness and thus the liability of the Worker`s Compensation Carrier.", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonDescription"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NoServiceLocation", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1261", "mean": "0.94", "stddev": "0.23", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "NoServiceLocation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ReasonDay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1261", "mean": "3.93", "stddev": "1.45", "min": "1", "max": "7", "missing": "0"}, "updatedLabel": "ReasonDay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ReasonMonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1261", "mean": "5.25", "stddev": "3.27", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "ReasonMonth"}, {"transformationsData": [{"feature_label": "Age_Bins", "transformation_label": "String Indexer"}], "feature": "Age_Bins", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "31-50", "max": "<=30", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Age_Bins"}, {"transformationsData": [{"feature_label": "Gender_Age_Bins", "transformation_label": "String Indexer"}], "feature": "Gender_Age_Bins", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender_Age_Bins"}, {"transformationsData": [{"feature_label": "Amount_Bins", "transformation_label": "String Indexer"}], "feature": "Amount_Bins", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Amount_Bins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PrimaryPayerAvailable", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1261", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "PrimaryPayerAvailable"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SecondaryPayerAvailable", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1261", "mean": "0.43", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "SecondaryPayerAvailable"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TertiaryPayerAvailable", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1261", "mean": "0.09", "stddev": "0.28", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "TertiaryPayerAvailable"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NumOfPayers", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1261", "mean": "1.51", "stddev": "0.65", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "NumOfPayers"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsPrimarySelfPay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1261", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "IsPrimarySelfPay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsPrimaryNonContracted", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1261", "mean": "0.03", "stddev": "0.18", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "IsPrimaryNonContracted"}, {"transformationsData": [{"feature_label": "label_resolution", "transformation_label": "String Indexer"}], "feature": "label_resolution", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1261", "mean": "27.46", "stddev": "10.62", "min": "0", "max": "64", "missing": "0", "distinct": "25"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "label_resolution"}, {"feature": "CPTCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "10.8", "stddev": "22.14", "min": "0.0", "max": "122.0", "missing": "0"}, "updatedLabel": "CPTCode_stringindexer_tra..."}, {"feature": "CptModifier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "0.8", "stddev": "2.35", "min": "0.0", "max": "21.0", "missing": "0"}, "updatedLabel": "CptModifier_stringindexer..."}, {"feature": "CptModifier2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1261", "mean": "0.0", "stddev": "0.07", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "CptModifier2_stringindexe..."}, {"feature": "pdiagnosis1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "82.71", "stddev": "101.06", "min": "0.0", "max": "384.0", "missing": "0"}, "updatedLabel": "pdiagnosis1_stringindexer..."}, {"feature": "pdiagnosis2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "91.11", "stddev": "107.85", "min": "0.0", "max": "405.0", "missing": "0"}, "updatedLabel": "pdiagnosis2_stringindexer..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1261", "mean": "0.44", "stddev": "0.5", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "DepartmentCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "2.51", "stddev": "4.31", "min": "0.0", "max": "29.0", "missing": "0"}, "updatedLabel": "DepartmentCategory_string..."}, {"feature": "SpecialityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "3.43", "stddev": "4.41", "min": "0.0", "max": "23.0", "missing": "0"}, "updatedLabel": "SpecialityCategory_string..."}, {"feature": "PrimaryPayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "1.08", "stddev": "2.22", "min": "0.0", "max": "15.0", "missing": "0"}, "updatedLabel": "PrimaryPayerCategory_stri..."}, {"feature": "PrimaryPayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "0.72", "stddev": "1.75", "min": "0.0", "max": "12.0", "missing": "0"}, "updatedLabel": "PrimaryPayerGroup_stringi..."}, {"feature": "PrimaryPayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "7.5", "stddev": "12.08", "min": "0.0", "max": "67.0", "missing": "0"}, "updatedLabel": "PrimaryPayerName_stringin..."}, {"feature": "PrimaryPayerSubGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "0.98", "stddev": "2.28", "min": "0.0", "max": "17.0", "missing": "0"}, "updatedLabel": "PrimaryPayerSubGroup1_str..."}, {"feature": "PrimaryPayerSubGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "3.13", "stddev": "4.05", "min": "0.0", "max": "22.0", "missing": "0"}, "updatedLabel": "PrimaryPayerSubGroup2_str..."}, {"feature": "SecondPayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "1.27", "stddev": "2.12", "min": "0.0", "max": "13.0", "missing": "0"}, "updatedLabel": "SecondPayerCategory_strin..."}, {"feature": "SecondPayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "1.09", "stddev": "1.79", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "SecondPayerGroup_stringin..."}, {"feature": "SecondPayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "3.28", "stddev": "6.79", "min": "0.0", "max": "46.0", "missing": "0"}, "updatedLabel": "SecondPayerName_stringind..."}, {"feature": "SecondPayerSubGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "0.94", "stddev": "1.54", "min": "0.0", "max": "13.0", "missing": "0"}, "updatedLabel": "SecondPayerSubGroup1_stri..."}, {"feature": "SecondPayerSubGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "1.59", "stddev": "3.05", "min": "0.0", "max": "20.0", "missing": "0"}, "updatedLabel": "SecondPayerSubGroup2_stri..."}, {"feature": "ServiceItemName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "10.8", "stddev": "22.14", "min": "0.0", "max": "122.0", "missing": "0"}, "updatedLabel": "ServiceItemName_stringind..."}, {"feature": "ServiceItemCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "8.82", "stddev": "19.98", "min": "0.0", "max": "113.0", "missing": "0"}, "updatedLabel": "ServiceItemCategory_strin..."}, {"feature": "ServiceLocation_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "0.1", "stddev": "0.48", "min": "0.0", "max": "6.0", "missing": "0"}, "updatedLabel": "ServiceLocation_stringind..."}, {"feature": "reasonType_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1261", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "reasonType_stringindexer_..."}, {"feature": "Reasoncode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "0.79", "stddev": "1.37", "min": "0.0", "max": "10.0", "missing": "0"}, "updatedLabel": "Reasoncode_stringindexer_..."}, {"feature": "Remarkcode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "1.23", "stddev": "2.6", "min": "0.0", "max": "21.0", "missing": "0"}, "updatedLabel": "Remarkcode_stringindexer_..."}, {"feature": "ReasonCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1261", "mean": "0.63", "stddev": "0.95", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "ReasonCategory_stringinde..."}, {"feature": "ReasonDescription_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "0.79", "stddev": "1.37", "min": "0.0", "max": "10.0", "missing": "0"}, "updatedLabel": "ReasonDescription_stringi..."}, {"feature": "Age_Bins_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1261", "mean": "0.5", "stddev": "0.73", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "Age_Bins_stringindexer_tr..."}, {"feature": "Gender_Age_Bins_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Gender_Age_Bins_stringind..."}, {"feature": "Amount_Bins_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Amount_Bins_stringindexer..."}, {"feature": "label_resolution_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1261", "mean": "3.18", "stddev": "4.01", "min": "0.0", "max": "24.0", "missing": "0"}, "updatedLabel": "label_resolution_stringin..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run eligdenresolutionrecommendationautomlHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["CPTCode_stringindexer", "CptModifier_stringindexer", "CptModifier2_stringindexer", "pdiagnosis1_stringindexer", "pdiagnosis2_stringindexer", "Age", "Gender_stringindexer", "Amount", "DepartmentCategory_stringindexer", "SpecialityCategory_stringindexer", "PrimaryPayerCategory_stringindexer", "PrimaryPayerGroup_stringindexer", "PrimaryPayerName_stringindexer", "PrimaryPayerSubGroup1_stringindexer", "PrimaryPayerSubGroup2_stringindexer", "SecondPayerCategory_stringindexer", "SecondPayerGroup_stringindexer", "SecondPayerName_stringindexer", "SecondPayerSubGroup1_stringindexer", "SecondPayerSubGroup2_stringindexer", "ServiceItemName_stringindexer", "ServiceItemCategory_stringindexer", "ServiceLocation_stringindexer", "reasonType_stringindexer", "TransactionReasonCodeID", "Reasoncode_stringindexer", "Remarkcode_stringindexer", "ReasonCategory_stringindexer", "ReasonDescription_stringindexer", "NoServiceLocation", "ReasonDay", "ReasonMonth", "Age_Bins_stringindexer", "Gender_Age_Bins_stringindexer", "Amount_Bins_stringindexer", "PrimaryPayerAvailable", "SecondaryPayerAvailable", "TertiaryPayerAvailable", "NumOfPayers", "IsPrimarySelfPay", "IsPrimaryNonContracted"], "label_resolution_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

